In [56]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%config InlineBackend. figure_format = 'retina'

# Part 2 (一)銷售概況分析

In [57]:
new_all_data = pd.read_csv('new_all_data.csv')

In [58]:
all_M_profit = new_all_data[['Year', 'Month', '單價', '成本', '獲利']]
all_M_profit

,Year,Month,單價,成本,獲利
0,2016,2016-01,643.195,394.800,248.395
1,2016,2016-01,391.510,225.365,166.145
2,2016,2016-01,713.930,416.185,297.745
3,2016,2016-01,557.655,337.225,220.430
4,2016,2016-01,628.390,366.835,261.555
...,...,...,...,...,...
353219,2019,2019-11,575.750,403.025,172.725
353220,2019,2019-11,575.750,403.025,172.725
353221,2019,2019-11,575.750,403.025,172.725
353222,2019,2019-11,575.750,403.025,172.725


In [59]:
all_M_profit = all_M_profit.groupby(['Year', 'Month'], as_index=False)['單價', '成本', '獲利'].sum()

C:\Users\chanm\AppData\Local\Temp\ipykernel_28280\86225934.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [60]:
all_M_profit = all_M_profit.rename(columns={'單價':'營收'})

In [61]:
# 只取2018,2019年
profit_M_18_19 = all_M_profit[(all_M_profit['Year'] == 2018)|(all_M_profit['Year'] == 2019)]

In [62]:
# 刪除不適合比較的月份
profit_M_18_19 = profit_M_18_19.drop([34,35,46])

In [63]:
profit_M_18_19 = profit_M_18_19.reset_index(drop=True)

In [64]:
profit_Y_18_19 = profit_M_18_19.groupby(['Year'], as_index=False)['營收', '成本', '獲利'].sum()

C:\Users\chanm\AppData\Local\Temp\ipykernel_28280\1591178153.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [65]:
profit_Y_18_19['成本佔比'] = round((profit_Y_18_19['成本']/profit_Y_18_19['營收'])*100, 1)

In [66]:
profit_Y_18_19['獲利佔比'] = round((profit_Y_18_19['獲利']/profit_Y_18_19['營收'])*100, 1)

In [67]:
profit_Y_18_19['Year'] = profit_Y_18_19['Year'].astype('str')

In [68]:
profit_Y_18_19 = profit_Y_18_19.T

In [69]:
profit_Y_18_19 = profit_Y_18_19.rename(columns={0:'2018年'})
profit_Y_18_19 = profit_Y_18_19.rename(columns={1:'2019年'})

In [70]:
profit_Y_18_19.drop('Year', axis = 0, inplace=True)

In [71]:
profit_Y_18_19 = profit_Y_18_19.astype(float)

In [72]:
profit_Y_18_19 = round(profit_Y_18_19)

In [73]:
profit_Y_18_19 = profit_Y_18_19.astype(int)

In [74]:
profit_Y_18_19['成長率'] = (profit_Y_18_19['2019年'] - profit_Y_18_19['2018年'])/profit_Y_18_19['2019年'] * 100
profit_Y_18_19['成長率'] = profit_Y_18_19['成長率'].astype(int)
profit_Y_18_19['成長率'] = round(profit_Y_18_19['成長率'], 1)


In [75]:
profit_Y_18_19 = profit_Y_18_19.rename(columns={'成長率':'成長率(%)'})

In [76]:
profit_Y_18_19.to_csv('profit_Y_18_19.csv', encoding='utf-8-sig')

In [77]:
profit_Y_18_19.loc['成本佔比':'獲利佔比','成長率(%)'] = np.nan 

In [78]:
profit_Y_18_19

,2018年,2019年,成長率(%)
營收,66794064,62321833,-7.0
成本,32612753,29465508,-10.0
獲利,34181310,32856325,-4.0
成本佔比,49,47,NaN
獲利佔比,51,53,NaN


In [79]:
profit_Y_18_19_fig = profit_Y_18_19.iloc[3:5, :2]
profit_Y_18_19_fig
fig_profit_Y_18_19 = px.pie(profit_Y_18_19_fig, values='2018年', names=['成本佔比','獲利佔比'], title='2018年度營收狀況', hole=.5)
fig_profit_Y_18_19.show()

In [80]:
fig_profit_Y_18_19 = px.pie(profit_Y_18_19_fig, values='2019年', names=['成本佔比',	'獲利佔比'], title='2019年度營收狀況', hole=.5)
fig_profit_Y_18_19.show()

In [81]:
all_M_profit.to_csv('all_M_profit.csv', encoding='utf-8-sig')
profit_M_18_19.to_csv('profit_M_18_19.csv', encoding='utf-8-sig')

In [82]:
all_M_profit

,Year,Month,營收,成本,獲利
0,2016,2016-01,6.341658e+06,4281043.410,2060614.185
1,2016,2016-02,4.425430e+06,2737852.460,1687577.535
2,2016,2016-03,4.419311e+06,2605346.065,1813964.530
3,2016,2016-04,5.906345e+06,2837147.950,3069196.585
4,2016,2016-05,7.040271e+06,3354046.430,3686224.570
5,2016,2016-06,7.713770e+06,3850444.920,3863325.270
6,2016,2016-07,6.809492e+06,3423455.560,3386036.745
7,2016,2016-08,6.465510e+06,3257519.475,3207990.170
8,2016,2016-09,5.954097e+06,3203848.060,2750249.180
9,2016,2016-10,5.305048e+06,2827649.720,2477397.965


In [83]:
profit_M_18_19

,Year,Month,營收,成本,獲利
0,2018,2018-01,6.321987e+06,3231230.730,3090755.955
1,2018,2018-02,6.662086e+06,3418726.185,3243359.315
2,2018,2018-03,6.150759e+06,3086844.145,3063914.490
3,2018,2018-04,7.607202e+06,3564494.570,4042707.585
4,2018,2018-05,6.696708e+06,3091469.885,3605237.930
5,2018,2018-06,5.369112e+06,2398071.130,2971041.080
6,2018,2018-07,7.785941e+06,3823540.945,3962400.330
7,2018,2018-08,6.690658e+06,3533407.360,3157250.145
8,2018,2018-09,5.672814e+06,2672049.170,3000764.585
9,2018,2018-10,7.836798e+06,3792919.270,4043878.825


# Part 2 (二)毛利率分析
產出2019年高GPM的data_2019銷售資料

In [84]:
import pandas as pd
import datetime as dt
import numpy as np
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [85]:
all_data = pd.read_csv('new_all_data.csv')

In [86]:
data_2019 = all_data[all_data['Year']==2019]

In [87]:
# 刪除兩筆售價為0的訂單(導致GPM = -inf)
# df.index[條件].tolist()
all_data.index[all_data['單價'] == 0].tolist()
all_data = all_data.drop(index = [285228, 313012])

In [88]:
high_GPM_2019 = (data_2019.groupby(['產品', '單價', '成本', '獲利', 'GPM'], as_index=False)['order_count'].count()).sort_values('order_count', ascending=False)

In [89]:
high_GPM_2019

,產品,單價,成本,獲利,GPM,order_count
980,產品13-12,985.355,450.730,534.625,54.3,638
2346,產品20-12,985.355,450.730,534.625,54.3,532
2344,產品20-12,985.355,432.635,552.720,56.1,529
1029,產品13-3,985.355,432.635,552.720,56.1,515
978,產品13-12,985.355,432.635,552.720,56.1,501
...,...,...,...,...,...,...
2845,產品26-11,1135.050,470.470,664.580,58.6,1
2847,產品26-12,820.855,401.380,419.475,51.1,1
2855,產品26-12,1061.025,383.285,677.740,63.9,1
2856,產品26-12,1061.025,401.380,659.645,62.2,1


In [90]:
# 'GPM_Rank'欄
high_GPM_2019 = high_GPM_2019.sort_values('GPM', ascending=False)
high_GPM_2019['GPM_Rank'] = range(1, len(high_GPM_2019)+1)

In [91]:
high_GPM_2019

,產品,單價,成本,獲利,GPM,order_count,GPM_Rank
1298,產品14-35,411.250,70.735,340.515,82.8,27,1
1297,產品14-35,327.355,70.735,256.620,78.4,8,2
4251,產品43-35,491.855,116.795,375.060,76.3,6,3
4340,產品45-176,1011.675,245.105,766.570,75.8,6,4
4344,產品45-177,1011.675,245.105,766.570,75.8,15,5
...,...,...,...,...,...,...,...
5721,產品8-88,394.800,449.085,-54.285,-13.7,1,6155
5714,產品8-73,394.800,449.085,-54.285,-13.7,2,6156
5715,產品8-73,394.800,450.730,-55.930,-14.2,5,6157
5722,產品8-88,394.800,450.730,-55.930,-14.2,20,6158


In [92]:

# 'Value'欄 = price * count_order 
high_GPM_2019['Value'] = high_GPM_2019['單價'] * high_GPM_2019['order_count']
high_GPM_2019['Value'] = high_GPM_2019['Value'].astype(int)


In [93]:
# 'value_rank'欄
high_GPM_2019 = high_GPM_2019.sort_values('Value', ascending=False)
high_GPM_2019['Value_Rank'] = range(1, len(high_GPM_2019)+1)


In [94]:
# 'Value_cunsum'欄
high_GPM_2019['Value_cunsum'] =high_GPM_2019['Value'].cumsum()

In [95]:
# '%_total_Value'欄
high_GPM_2019['%_total_Value'] = round(100*(high_GPM_2019['Value_cunsum']/data_2019['單價'].sum()), 1)


In [96]:
high_GPM_2019 = high_GPM_2019[['產品', '單價', '成本', '獲利', 'order_count', 'GPM', 'GPM_Rank', 'Value', 'Value_Rank', 'Value_cunsum', '%_total_Value']]
high_GPM_2019.reset_index(drop=True, inplace=True)
high_GPM_2019

,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
0,產品13-12,985.355,450.730,534.625,638,54.3,3454,628656,1,628656,1.0
1,產品1-35,1628.550,689.255,939.295,345,57.7,1983,561849,2,1190505,1.9
2,產品20-12,985.355,450.730,534.625,532,54.3,3471,524208,3,1714713,2.7
3,產品20-12,985.355,432.635,552.720,529,56.1,2579,521252,4,2235965,3.5
4,產品8-35,1460.760,1233.750,227.010,355,15.5,6133,518569,5,2754534,4.3
...,...,...,...,...,...,...,...,...,...,...,...
6154,產品69-35,254.975,152.985,101.990,1,40.0,5419,254,6155,63365958,100.0
6155,產品42-35,246.750,62.510,184.240,1,74.7,10,246,6156,63366204,100.0
6156,產品25-35,246.750,111.860,134.890,1,54.7,3160,246,6157,63366450,100.0
6157,產品8-36,164.500,106.925,57.575,1,35.0,5654,164,6158,63366614,100.0


In [97]:
high_GPM_2019.to_csv('high_GPM_2019.csv', encoding = 'utf-8-sig')

# 以大於55.4(四分位數(50%))的GM篩選

In [98]:
np.percentile(high_GPM_2019['GPM'], (25, 50, 75))

array([47.2, 55.4, 59.2])

In [99]:
fig = px.box(data_2019['GPM'], x="GPM")
fig.show()
#sns.boxplot(x = data_2019['GPM'])

In [100]:
high_GPM_2019_select = high_GPM_2019[high_GPM_2019['GPM'] > 56.1] #(np.percentile(high_GPM_2019['GPM'], (75)))]
# high_GPM_2019 = high_GPM_2019[high_GPM_2019['Value_Rank'] < 100]
high_GPM_2019_select = high_GPM_2019_select.sort_values('GPM', ascending=False)

high_GPM_2019_select

,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
911,產品14-35,411.250,70.735,340.515,27,82.8,1,11103,912,50030955,79.0
2537,產品14-35,327.355,70.735,256.620,8,78.4,2,2618,2538,58772182,92.7
2459,產品43-35,491.855,116.795,375.060,6,76.3,3,2951,2460,58553687,92.4
1430,產品45-176,1011.675,245.105,766.570,6,75.8,4,6070,1431,54314838,85.7
908,產品45-174,1011.675,245.105,766.570,11,75.8,6,11128,909,49997604,78.9
...,...,...,...,...,...,...,...,...,...,...,...
3770,產品23-16,689.255,299.390,389.865,2,56.6,2283,1378,3771,61116599,96.4
354,產品55-129,967.260,422.765,544.495,38,56.3,2426,36755,355,39101477,61.7
1734,產品11-35,970.550,424.410,546.140,5,56.3,2427,4852,1735,55955242,88.3
464,產品2-113,1737.120,761.635,975.485,15,56.2,2428,26056,465,42515680,67.1


In [101]:
print('2019年GPM大於55.4的訂單數佔2019年訂單數'+str(round((high_GPM_2019_select['order_count'].sum()) / len(data_2019) *100, 1)) + '%')


2019年GPM大於55.4的訂單數佔2019年訂單數34.3%


In [102]:
np.percentile(data_2019['單價'], (25, 50, 75))

array([ 820.855,  985.355, 1061.025])

# 找出行銷策略優先採納的高GPM商品

In [103]:
# 來看high_GPM_2019_select['Value']前30名的數值為何
high_GPM_2019_select.sort_values('Value', ascending=False).head(50)
# 暫且設標準為100000營業額以上

,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
1,產品1-35,1628.550,689.255,939.295,345,57.7,1983,561849,2,1190505,1.9
11,產品14-135,1135.050,378.350,756.700,304,66.7,109,345055,12,5850696,9.2
14,產品3-35,1135.050,406.315,728.735,276,64.2,247,313273,15,6843981,10.8
16,產品1-66,1135.050,430.990,704.060,264,62.0,574,299653,17,7447123,11.8
21,產品14-132,970.550,383.285,587.265,280,60.5,1160,271754,22,8866974,14.0
24,產品3-97,1464.050,618.520,845.530,172,57.8,1955,251816,25,9630300,15.2
28,產品20-12,1061.025,450.730,610.295,226,57.5,2142,239791,29,10607271,16.7
31,產品20-12,1061.025,432.635,628.390,219,59.2,1533,232364,32,11312604,17.9
33,產品14-136,1299.550,378.350,921.200,174,70.9,21,226121,34,11765356,18.6
35,產品20-12,1135.050,432.635,702.415,196,61.9,663,222469,36,12213471,19.3


In [104]:
# 從GPM前50%的商品，進一步篩選推薦清單: high_GPM_2019['Value'] > 100000 的產品
recommend_list =  high_GPM_2019_select[(high_GPM_2019_select['Value'] > 100000)] 

In [105]:
recommend_list = recommend_list.sort_values('Value', ascending=False)

In [106]:
# 加工:重新計算累績價值、以及佔所有產品價值的幾％、RESET_INDEX
recommend_list['Value_cunsum'] = recommend_list['Value'].cumsum()

In [107]:
recommend_list['%_total_Value'] = round(100*(recommend_list['Value_cunsum']/data_2019['單價'].sum()), 1)

In [108]:
recommend_list.reset_index(drop= True, inplace = True)


In [109]:
print(len(recommend_list['產品'].unique()))
recommend_list
# 以提高2019年GPM為目標的推薦產品清單，佔累積營業額約10%的商品組合，共有32項商品

32


,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
0,產品1-35,1628.550,689.255,939.295,345,57.7,1983,561849,2,561849,0.9
1,產品14-135,1135.050,378.350,756.700,304,66.7,109,345055,12,906904,1.4
2,產品3-35,1135.050,406.315,728.735,276,64.2,247,313273,15,1220177,1.9
3,產品1-66,1135.050,430.990,704.060,264,62.0,574,299653,17,1519830,2.4
4,產品14-132,970.550,383.285,587.265,280,60.5,1160,271754,22,1791584,2.8
5,產品3-97,1464.050,618.520,845.530,172,57.8,1955,251816,25,2043400,3.2
6,產品20-12,1061.025,450.730,610.295,226,57.5,2142,239791,29,2283191,3.6
7,產品20-12,1061.025,432.635,628.390,219,59.2,1533,232364,32,2515555,4.0
8,產品14-136,1299.550,378.350,921.200,174,70.9,21,226121,34,2741676,4.3
9,產品20-12,1135.050,432.635,702.415,196,61.9,663,222469,36,2964145,4.7


In [110]:
recommend_list.to_csv('recommend_list.csv', encoding = 'utf-8-sig')